# The chilling adventures of Sabrina
## Problem definition
Spellman´s Ltd is a company that manufactures chilling soft drinks. They want to manufacture two types of drinks A, and B. Both beverages use a semi-elaborate C, another expensive ingredient D and other ingredients that are not relevant for production planning. Sabrina is a young student of engineering and management doing an intership at Spellman´s. She needs to formulate a Continuous Linear Program to configure the optimal daily production plan for the company.

The selling price of drink A is 3€/liter and the selling price of drink B is 2€/liter.

1 liter of drink A uses 3 grams of ingredient D. A liter of drink B uses 1 gram of ingredient D. There are only 3 grams of ingredient D available per day.

The factory only has one mixer to elaborate both drink types and the semi-elaborate. It takes 1 hour to process a liter of drink A, 1 hour to process 1 liter of drink B, and 1 hour to process 1cl of semi-elaborate C. The mixer is available 6 hours per day.

Drink A uses 2cl of semi-elaborate C and drink B uses 1cl of semi-elaborate C. The company has 3cl of semi-elaborate C plus the amount they decide to produce available per day.

Write a Continuous Linear Problem to help Sabrina design the optimal production plan that maximises revenues for the company.

**Decision variables**:
- $x_A$: Production of drink A in liters
- $x_B$: Production of drink B in liters
- $x_C$: Production of semi-elaborate C in centiliters

$x_A, x_C, x_B \in \mathbb{R}$

**Objective function**

$\max z = 3*x_A + 2*x_B $

z is the profit in euros. 

**Constraints**
s.t.

- Availability of ingredient D in grams:

$3*x_A + x_B \leq 3$

- Availability of mixer in hours: 

$x_A + x_B + x_C \leq 6$

- Availability of semi-elaborate C in centiliters: 

$2*x_A +  x_B -x_C  \leq 3$

- Logical constraint

$ x_A, x_C, x_B \geq 0$

In [2]:
import pulp
import pandas as pd
model = pulp.LpProblem("Chilling adventures of Sabrina", pulp.LpMaximize)
product_types = ['A', 'B', 'C']

units = pulp.LpVariable.dicts("units",
                                     (i for i in product_types),
                                     lowBound=0,
                                     cat='Continuous')

# Technological Coefficients:
unit_profits = [3, 2, 0]

# Objective Function
model += (
    pulp.lpSum([
        unit_profits[i] * units[product_types[i]]
        for i in range(len(product_types))])
)

# Availability of ingredient
model += 3*units['A'] + units['B'] <= 3, "Availability of ingredient A"

# Availability of mixer hours
model += units['A'] + units['B'] + units['C'] <= 6, "Availability of mixer hours"

# Availability of semi-elaborate
model += 2*units['A'] + units['B'] - units['C'] <= 6, "Availability of semi-elaborate"

model.solve(solver=pulp.GUROBI(msg = 0))

print(pulp.LpStatus[model.status])

total_profit = pulp.value(model.objective)
print("Total profit is %0.2f €"%total_profit)

print("The following table shows the decision variables: ")
var_df = pd.DataFrame.from_dict(units, orient="index", 
                                columns = ["Variables"])
var_df["Solution (GRB)"] = var_df["Variables"].apply(lambda item: "{:.2f}".format(item.solverVar.X))
var_df["Reduced cost (GRB)"] = var_df["Variables"].apply(lambda item: "{:.2f}".format(item.solverVar.RC))
var_df["Objective Coefficient (GRB)"] = var_df["Variables"].apply(lambda item: "{:.2f}".format(item.solverVar.Obj))
var_df["Objective Lower bound (GRB)"] = var_df["Variables"].apply(lambda item: "{:.2f}".format(item.solverVar.SAObjLow) if item.solverVar.SAObjLow > -0.1 else "-Inf" )
var_df["Objective Upper bound (GRB)"] = var_df["Variables"].apply(lambda item: "{:.2f}".format(item.solverVar.SAObjUp) if item.solverVar.SAObjUp != item.solverVar.UB else "Inf")


print(var_df.to_markdown())


const_dict = dict(model.constraints)
con_df = pd.DataFrame.from_records(list(const_dict.items()), exclude=["Expression"], columns=["Constraint", "Expression"])
con_df["Slack"]=con_df["Constraint"].apply(lambda item: "{:.2f}".format(const_dict[item].solverConstraint.Slack))
con_df["Shadow Price"]=con_df["Constraint"].apply(lambda item: "{:.2f}".format(const_dict[item].solverConstraint.Pi))
con_df["Right Hand Side"]=con_df["Constraint"].apply(lambda item: "{:.2f}".format(const_dict[item].solverConstraint.RHS))
con_df["Min RHS"]=con_df["Constraint"].apply(lambda item: "{:.2f}".format(const_dict[item].solverConstraint.SARHSLow) )
con_df["Max RHS"]=con_df["Constraint"].apply(lambda item: "{:.2f}".format(const_dict[item].solverConstraint.SARHSUp) if const_dict[item].solverConstraint.SARHSUp < 1e10 else "Inf" )


print("The following table shows the constraints: ")
print(con_df.to_markdown())

Optimal
Total profit is 6.00 €
The following table shows the decision variables: 
|    | Variables   |   Solution (GRB) |   Reduced cost (GRB) |   Objective Coefficient (GRB) |   Objective Lower bound (GRB) |   Objective Upper bound (GRB) |
|:---|:------------|-----------------:|---------------------:|------------------------------:|------------------------------:|------------------------------:|
| A  | units_A     |                0 |                   -3 |                             3 |                          -inf |                           6   |
| B  | units_B     |                3 |                    0 |                             2 |                             1 |                         inf   |
| C  | units_C     |                3 |                    0 |                             0 |                            -0 |                           1.5 |
The following table shows the constraints: 
|    | Constraint                     |   Slack |   Shadow Price |   Right Hand